In [83]:
import pandas as pd
import numpy as np
df = pd.read_csv('bank_balanced.csv')
print(df.shape)
print(df.info())
df.head()


(10578, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10578 entries, 0 to 10577
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        10578 non-null  int64 
 1   job        10578 non-null  object
 2   marital    10578 non-null  object
 3   education  10578 non-null  object
 4   default    10578 non-null  object
 5   balance    10578 non-null  int64 
 6   housing    10578 non-null  object
 7   loan       10578 non-null  object
 8   contact    10578 non-null  object
 9   day        10578 non-null  int64 
 10  month      10578 non-null  object
 11  duration   10578 non-null  int64 
 12  y          10578 non-null  object
dtypes: int64(4), object(9)
memory usage: 1.0+ MB
None


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,y
0,25,student,single,unknown,no,480,no,no,cellular,8,sep,140,yes
1,28,services,married,secondary,no,510,yes,yes,cellular,8,jul,1767,yes
2,39,management,single,tertiary,no,0,yes,no,cellular,22,may,271,yes
3,45,housemaid,married,secondary,no,360,no,no,cellular,3,nov,215,yes
4,53,retired,married,tertiary,no,1691,no,no,unknown,13,may,916,no


In [84]:
df = df.drop(['day', 'month', 'contact', 'job', 'balance'], axis=1)
df.head()
#1. Read data and select the fields that I think can help with better prediction.

,age,marital,education,default,housing,loan,duration,y
0,25,single,unknown,no,no,no,140,yes
1,28,married,secondary,no,yes,yes,1767,yes
2,39,single,tertiary,no,yes,no,271,yes
3,45,married,secondary,no,no,no,215,yes
4,53,married,tertiary,no,no,no,916,no


In [85]:
from collections import Counter
m = Counter(df['marital'])
e = Counter(df['education'])
d = Counter(df['default'])
h = Counter(df['housing'])
l = Counter(df['loan'])
y = Counter(df['y'])

df['marital_num'] = df['marital'].apply(lambda x: list(m.keys()).index(x))
df['default_num'] = df['default'].apply(lambda x: list(d.keys()).index(x))
df['housing_num'] = df['housing'].apply(lambda x: list(h.keys()).index(x))
df['load_num'] = df['loan'].apply(lambda x: list(l.keys()).index(x))
df['y_num'] = df['y'].apply(lambda x: list(y.keys()).index(x))
df['education_num'] = df['education'].apply(lambda x: list(e.keys()).index(x))
df.head()


,age,marital,education,default,housing,loan,duration,y,marital_num,default_num,housing_num,load_num,y_num,education_num
0,25,single,unknown,no,no,no,140,yes,0,0,0,0,0,0
1,28,married,secondary,no,yes,yes,1767,yes,1,0,1,1,0,1
2,39,single,tertiary,no,yes,no,271,yes,0,0,1,0,0,2
3,45,married,secondary,no,no,no,215,yes,1,0,0,0,0,1
4,53,married,tertiary,no,no,no,916,no,1,0,0,0,1,2


In [86]:
df['education_num'].replace(0, np.random.randint(1, 4), inplace=True)
df = df.drop(['marital', 'education', "default",
             "housing", "loan", "y"], axis=1)
df.head()
#2. Change categorical values to numeric and scale the values.

,age,duration,marital_num,default_num,housing_num,load_num,y_num,education_num
0,25,140,0,0,0,0,0,1
1,28,1767,1,0,1,1,0,1
2,39,271,0,0,1,0,0,2
3,45,215,1,0,0,0,0,1
4,53,916,1,0,0,0,1,2


In [87]:
df_y = df[['y_num']]
df_x = df.drop(['y_num'], axis=1)
print(df_y.shape, df_x.shape)
#3. Divide the data into training set and test sets. 

(10578, 1) (10578, 7)


In [88]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    df_x, df_y, test_size=0.2, random_state=0)
clf = RandomForestClassifier(oob_score=True, random_state=0)
clf = clf.fit(x_train, y_train)
#4. Choose Random Forest as machine learning algorithms to train a predictive model using training set. 

<ipython-input-88-0dbd070636a1>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = clf.fit(x_train, y_train)


In [89]:
from sklearn.metrics import accuracy_score, confusion_matrix
y_train_pred = clf.predict(x_train)
train_acc = accuracy_score(y_train, y_train_pred)
print('Train Accuracy:', round(train_acc * 100, 2))
y_pred = clf.predict(x_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print('Accuracy:', round(acc*100, 2))
print('Confusion Matrix:')
print(cm)
#5. Evaluate the model with accuracy and confusion matrix.

Train Accuracy: 99.49
Accuracy: 77.22
Confusion Matrix:
[[846 225]
 [257 788]]
